In [3]:
import math

In [1]:
# f(x,y,z) = (x + y)z
x = -2
y = 5
z = -4

q = x + y
f = q * z

dfdz = q
dfdq = z

dfdy = 1 * dfdq 
dfdx = 1 * dfdq

print ("q is %d" % q)
print ("z is %d" % z)

print ("dfdz is %d" % dfdz)
print ("dfdq is %d" % dfdq)

print ("dfdy is %d" % dfdy)
print ("dfdx is %d" % dfdx)

q is 3
z is -4
dfdz is 3
dfdq is -4
dfdy is -4
dfdx is -4


In [4]:
# f(w,x) = 1 / (1 + math.exp(- (w0*x0 + w1*x1 + w2)))

w = [2, -3, -3]
x = [-1, -2]

# forward pass
weightedsum = w[0] * x[0] + w[1] * x[1] + w[2]
f = 1 / (1 + math.exp(-weightedsum))

# backward pass
dfdweightedsum = (1 - f) * f # gradient on the weighted sum, because that is the sigmoid activation derivation
dx = [w[0] * dfdweightedsum, w[1] * dfdweightedsum]
dw = [x[0] * dfdweightedsum, x[1]* dfdweightedsum, 1.0 * dfdweightedsum]

print (dx)
print (dw)

[0.3932238664829637, -0.5898357997244456]
[-0.19661193324148185, -0.3932238664829637, 0.19661193324148185]


In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

SEED = 42
tf.set_random_seed(SEED)
np.random.seed(SEED)

In [2]:
def init_weights(name,shape):
    """ Weight initialization """
    # we almost always want the weights to be gaussian distribution
    # but we will play with the normal method and see what are the effects.
    return tf.get_variable(name, shape=shape, dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
    

In [3]:
def get_iris_data():
    """ Read iris data and split """
    iris = datasets.load_iris()
    input_data = iris["data"]
    labels = iris["target"]
    
    # convert to one-hot vectors
    oh_y = np.eye(len(np.unique(labels)))[labels]
    # split the data set into train and test set because YOLO
    return train_test_split(input_data, oh_y, test_size=0.33, random_state=SEED)

In [4]:
train_X, test_X, train_Y, test_Y = get_iris_data()
print("training shape has %s " % str(train_X.shape))
print("testing shape has %s" % str(test_X.shape))

training shape has (100, 4) 
testing shape has (50, 4)


In [10]:
# Input neurons should be 4 , beacause the data set x features are 4.
# output neruons are 3 , because iris has 3 classes
input_neurons = 4
hidden_neurons = 5
hidden_neurons2 = 5
output_neurons = 3

tf.reset_default_graph()

# init our data_placeholder 
# init to None for tf to determine at runtime 
input_x = tf.placeholder(tf.float32, shape=[None, input_neurons], name='inputs')
label_y = tf.placeholder(tf.float32, shape=[None, output_neurons], name='ouputs')

# initialise our Variables
w1 = init_weights('w1',[input_neurons, hidden_neurons]) # first layer -> hidden
b1 = init_weights('b1',[hidden_neurons])

w2 = init_weights('w2',[hidden_neurons, hidden_neurons2]) # hidden -> ouput
b2 = init_weights('b2',[hidden_neurons2])

w3 = init_weights('w3',[hidden_neurons2, output_neurons]) # first layer -> hidden
b3 = init_weights('b3',[output_neurons])

# forward pass
hidden_ouput = tf.nn.tanh(tf.add(tf.matmul(input_x, w1), b1))
hidden2_ouput = tf.nn.tanh(tf.add(tf.matmul(hidden_ouput, w2), b2))
final_output = tf.add(tf.matmul(hidden2_ouput, w3), b3)

# backward pass
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=label_y, logits=final_output))
update_op = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

# define out prediction op
predict_op = tf.argmax(final_output, axis=1)

# start init all the variables
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(5000):
    # batch training
    sess.run(update_op, feed_dict={input_x: train_X, label_y: train_Y})
    
    if epoch % 100 == 0 :
        prediction = sess.run(predict_op, feed_dict={input_x: train_X, label_y: train_Y})
        test_prediction = sess.run(predict_op, feed_dict={input_x: test_X, label_y: test_Y})
        train_accuracy = np.mean(np.argmax(train_Y, axis=1) == prediction)
        test_accuracy = np.mean(np.argmax(test_Y, axis=1) == test_prediction)
        print ("Epoch = %d, train accuracy = %.2f%%, test accuracy=%.2f%%" 
               % (epoch +1, 100 * train_accuracy, 100 * test_accuracy))

Epoch = 1, train accuracy = 31.00%, test accuracy=38.00%
Epoch = 101, train accuracy = 32.00%, test accuracy=38.00%
Epoch = 201, train accuracy = 49.00%, test accuracy=48.00%
Epoch = 301, train accuracy = 64.00%, test accuracy=68.00%
Epoch = 401, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 501, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 601, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 701, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 801, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 901, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 1001, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 1101, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 1201, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 1301, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 1401, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 1501, train accuracy = 66.00%, test accuracy=68.00%
Epoch = 1601, train accuracy = 66.00%, test accuracy